[Bokeh Web UI](http://localhost:8787)

In [ ]:
from pprint import pprint

from dask.distributed import Client
client = Client('dask-scheduler:8786')

In [ ]:
client

In [ ]:
pprint(client.scheduler_info())

In [ ]:
client.list_datasets()

# Dask DataFrame

In [ ]:
import dask.dataframe as dd

In [ ]:
!ls data/*.csv.gz

In [ ]:
df = dd.read_csv("/notebooks/data/food_inspection_201*", 
                 blocksize=None, 
                 compression='gzip', 
                 parse_dates=['Inspection Date'])

In [ ]:
df

In [ ]:
df.head()

In [ ]:
failed = df[df.Results == 'Fail']

In [ ]:
years = failed.year.unique()
years = years.compute()
years = years.values.tolist()

In [ ]:
divisions = years + [2016]
divisions

In [ ]:
failed = failed.set_partition('year', divisions=divisions)

In [ ]:
failed.known_divisions

In [ ]:
failed.head()

In [ ]:
def violators(df, n=3):
    size = lambda x: len(x)
    return (df.groupby('Address').
            apply(lambda x: len(x)).
            nlargest(n=n).
            to_frame(name='violations').
            reset_index())

In [ ]:
result = (failed.groupby(failed.index).
          apply(violators, 
                meta=[('Address', object), 
                      ('Violations', int)]))
result

In [ ]:
worst = result.compute()

In [ ]:
worst.index.names = ['Year', 'Offender']

In [ ]:
worst.sort_index(level=0)